### This jupyter notebook is to match face of a given photo with all the faces from photos folder 

In [3]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

In [4]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('labeled_faces/test') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


C:\Users\ashwi\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


### Saving data into data.pt file

In [169]:
data[0][0]

tensor([[ 2.2244e-02, -5.3660e-02, -6.0101e-02, -8.9254e-02,  1.1569e-02,
         -5.1270e-03, -1.6831e-02, -4.4577e-03,  7.2051e-02, -6.0032e-02,
         -7.4484e-02,  4.7916e-02,  5.4933e-02, -1.6775e-02, -6.7788e-03,
          2.1315e-02, -2.4184e-02, -3.9030e-02,  5.4845e-02,  1.9401e-02,
          3.6292e-02, -2.7346e-02,  4.0775e-02,  1.6314e-02,  8.5889e-02,
          1.1859e-02, -3.6736e-02, -6.6740e-02,  8.9234e-03,  3.5632e-03,
         -1.4481e-02, -2.7409e-02, -6.1107e-02,  2.7576e-02,  7.2388e-02,
          1.2719e-02,  2.6347e-02, -3.0697e-02, -5.1767e-02,  5.9949e-02,
          1.2951e-02,  3.0715e-02, -9.9562e-02,  2.1229e-02, -5.0728e-02,
          1.4605e-02, -2.1627e-03,  1.3950e-02, -6.5230e-02, -4.6013e-03,
         -3.7671e-02, -4.4239e-02, -7.3892e-02,  3.3165e-02, -1.0171e-02,
          7.4949e-02,  2.1729e-02,  8.4439e-04,  9.1955e-03, -1.3587e-02,
          5.1921e-02,  9.0390e-02, -5.3600e-02,  1.3235e-02,  1.9441e-02,
          5.5693e-02,  2.9380e-02, -2.

In [29]:
data = [embedding_list, name_list]
torch.save(embedding_list, 'data.npy') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [168]:
img,lbl = mtcnn('labeled_faces/train/priyanka_chopra/2.1.jpg', return_prob=True)

AttributeError: 'str' object has no attribute 'size'

In [10]:
from PIL import Image
def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))


result = face_match('labeled_faces/train/priyanka_chopra/2.1.jpg', 'data.pt')

print('Face matched with: ',result[0], 'With distance: ',result[1])

RuntimeError: number of dims don't match in permute

# ROSA's IMPLEMENTATION

In [140]:
import cv2
import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models,transforms,utils,datasets
from torchvision.transforms import ToTensor, Resize, Normalize
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
from facenet_pytorch import MTCNN
from PIL import Image
from torchvision import models
import time,os,copy

In [141]:
TRAIN_PATH = 'labeled_faces/train/'

VAL_PATH = 'labeled_faces/test/'

In [115]:
image_datasets['train']

220

In [116]:
# Create a dataset from ImageFolder

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'labeled_faces/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
                  
# Save as npy files

train_data = []

for img, lbl in image_datasets['train']:
    train_data.append([np.array(img), np.array(lbl)])
    
train_data_np = np.array(train_data)
np.save('train_data' + '.npy', train_data_np)

val_data = []

for img, lbl in image_datasets['test']:
    val_data.append([np.array(img), np.array(lbl)])
    
val_data_np = np.array(val_data)
np.save('val_data' + '.npy', val_data_np)

<ipython-input-116-9b63e3a2d681>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_data_np = np.array(train_data)
<ipython-input-116-9b63e3a2d681>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_data_np = np.array(val_data)


In [117]:
class Data(Dataset):
    def __init__(self, datafile):
        self.datafile = datafile

    def __len__(self):
        return self.datafile.shape[0]

    def __getitem__(self, idx):
        img = self.datafile[idx][0]
        y = int(self.datafile[idx][1])
        if y == 0:
            lbl = torch.tensor([1.0,0.0,0.0])
        elif y == 1:
            lbl = torch.tensor([0.0,1.0,0.0])
        else:
            lbl = torch.tensor([0.0,0.0,1.0])

        return img, lbl

In [118]:
train_data = np.load('train_data.npy',allow_pickle=True)
val_data = np.load('val_data.npy',allow_pickle=True)

In [119]:
train_data.shape

(220, 2)

In [120]:
val_data.shape

(48, 2)

In [121]:
train_data_class = Data(train_data)
val_data_class = Data(val_data)

In [122]:
train_loader = DataLoader(train_data_class,batch_size=32,shuffle=True)
val_loader = DataLoader(val_data_class,batch_size=32,shuffle=True)

In [152]:
val_data_class[0]

(array([[[-1.9637812 , -1.9980307 , -1.9980307 , ..., -1.5699118 ,
          -1.6041614 , -1.6726604 ],
         [-1.980906  , -1.980906  , -1.980906  , ..., -1.7411594 ,
          -1.7754089 , -1.5185376 ],
         [-1.9980307 , -1.980906  , -1.9637812 , ..., -1.6555357 ,
          -1.6555357 , -1.6555357 ],
         ...,
         [-1.3301654 , -1.2959158 , -1.2102921 , ..., -1.8096584 ,
          -1.4671633 , -1.2959158 ],
         [-1.2445416 , -1.2274169 , -1.1931673 , ..., -1.6897851 ,
          -1.7754089 , -1.5356624 ],
         [-1.1931673 , -1.2102921 , -1.1931673 , ..., -1.7582842 ,
          -1.7582842 , -1.7582842 ]],
 
        [[-1.8781512 , -1.9131652 , -1.9131652 , ..., -1.4754901 ,
          -1.5105042 , -1.5805322 ],
         [-1.8956583 , -1.8956583 , -1.8956583 , ..., -1.6505601 ,
          -1.6855742 , -1.4229691 ],
         [-1.9131652 , -1.8956583 , -1.8781512 , ..., -1.5630252 ,
          -1.5630252 , -1.5630252 ],
         ...,
         [-1.230392  , -1.1953781

In [123]:
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': train_data.shape[0], 'val': val_data.shape[0]}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Building Model

In [124]:
## Model as feature extractor
model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Sequential(nn.Linear(num_ftrs, 3), torch.nn.Sigmoid())

criterion = nn.MSELoss()

# Observe that only parameters of final layer are being optimized as opposed to before.
#optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [166]:
# Testing with one batch

img, lbl = next(iter(dataloaders['val']))

outputs = model_conv(img)
value, preds = torch.max(outputs, 1)
loss = criterion(outputs, lbl)

print(loss)

tensor(0.1173, grad_fn=<MseLossBackward0>)


# Training Model 

In [126]:
def train_model(model, optimizer,criterion, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 1.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    value, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == torch.argmax(labels,1)) #labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [127]:
model_conv = train_model(model_conv,  optimizer_conv, criterion,  exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 0.2249 Acc: 0.3818
val Loss: 0.2721 Acc: 0.2500

Epoch 1/24
----------
train Loss: 0.1976 Acc: 0.5227
val Loss: 0.2139 Acc: 0.3750

Epoch 2/24
----------
train Loss: 0.1670 Acc: 0.6727
val Loss: 0.2208 Acc: 0.5208

Epoch 3/24
----------
train Loss: 0.1488 Acc: 0.7545
val Loss: 0.1978 Acc: 0.6042

Epoch 4/24
----------
train Loss: 0.1314 Acc: 0.8000
val Loss: 0.1717 Acc: 0.7083

Epoch 5/24
----------
train Loss: 0.1191 Acc: 0.8455
val Loss: 0.1669 Acc: 0.7083

Epoch 6/24
----------
train Loss: 0.1068 Acc: 0.8682
val Loss: 0.1378 Acc: 0.7917

Epoch 7/24
----------
train Loss: 0.1005 Acc: 0.9182
val Loss: 0.1252 Acc: 0.8542

Epoch 8/24
----------
train Loss: 0.1046 Acc: 0.9000
val Loss: 0.1143 Acc: 0.8750

Epoch 9/24
----------
train Loss: 0.0982 Acc: 0.9136
val Loss: 0.1094 Acc: 0.8958

Epoch 10/24
----------
train Loss: 0.0940 Acc: 0.9364
val Loss: 0.1070 Acc: 0.9167

Epoch 11/24
----------
train Loss: 0.0936 Acc: 0.9182
val Loss: 0.1058 Acc: 0.9167

Ep

In [128]:
torch.save(model_conv.state_dict(),'models/model-resnet18-2.pth')

# Test phase

In [172]:
val_data_class[0][0].shape

(3, 224, 224)

In [177]:
transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [178]:
img1 = Image.open('labeled_faces/train/priyanka_chopra/2.1.jpg')

In [176]:
model_conv(val_data_class[0][0][0][0])

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: ([31;1mnumpy.ndarray[0m, [31;1mParameter[0m, [31;1mNoneType[0m, [31;1mtuple[0m, [31;1mtuple[0m, [31;1mtuple[0m, [32;1mint[0m)
